In [1]:
import torch
from rfm.nmfhtr import KernelModel
from rfm.kernels import laplacian

# Thiết lập thiết bị
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tạo dữ liệu ngẫu nhiên
n_samples = 1000  # Số lượng mẫu
n_features = 20   # Số lượng đặc trưng
n_classes = 3     # Số lượng lớp (đầu ra)

# Dữ liệu đầu vào (ngẫu nhiên)
X_train = torch.randn(n_samples, n_features).to(DEVICE)
X_val = torch.randn(n_samples // 10, n_features).to(DEVICE)

# Nhãn đầu ra (ngẫu nhiên, one-hot encoding)
y_train = torch.randint(0, n_classes, (n_samples,)).to(DEVICE)
y_train = torch.nn.functional.one_hot(y_train, num_classes=n_classes).float()

y_val = torch.randint(0, n_classes, (n_samples // 10,)).to(DEVICE)
y_val = torch.nn.functional.one_hot(y_val, num_classes=n_classes).float()

# Khởi tạo KernelModel với kernel Laplace
bandwidth = 1.0  # Tham số bandwidth của kernel Laplace
kernel_model = KernelModel(
    kernel_fn=lambda x, y: laplacian(x, y, bandwidth=bandwidth),
    centers=X_train,
    y_dim=n_classes,
    device=DEVICE
)

# Huấn luyện mô hình
print("Training KernelModel with Laplace kernel...")
# Huấn luyện mô hình
print("Training KernelModel with Laplace kernel...")
results = kernel_model.fit(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    epochs=10,          # Số lượng epochs
    mem_gb=4,           # Giới hạn bộ nhớ GPU
    n_subsamples=500,   # Số lượng mẫu con để tính toán
    bs=32,              # Batch size
    eta=0.1,            # Learning rate
    classification=True,
    verbose=True
)

# Dự đoán trên tập validation
print("Predicting on validation data...")
predictions = kernel_model.forward(X_val)

# Đánh giá độ chính xác
accuracy = (predictions.argmax(dim=1) == y_val.argmax(dim=1)).float().mean().item()
print(f"Validation Accuracy: {accuracy:.4f}")

c:\Users\hoang\anaconda3\envs\clean_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training KernelModel with Laplace kernel...
Training KernelModel with Laplace kernel...
Using NMF-based projection with rank=3, eta=0.0031, bs=32


  0%|          | 0/31 [00:00<?, ?it/s]

Using NMF projection
grad shape: torch.Size([33, 3]), kmat shape: torch.Size([33, 500])
W_nmf shape: torch.Size([3, 3]), H_nmf shape: torch.Size([3, 500]), S_nmf shape: torch.Size([3, 3])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x500 and 33x3)